In [1]:
# 0) duomenys
import pandas as pd
from sklearn.model_selection import train_test_split

path = "diamonds.csv"
df = pd.read_csv(path)

# kiekybiniai stulpeliai
qCols = ['carat', 'depth', 'table', 'price', 'x', 'y','z']

# kokybiniai stulpeliai
ordCols = ['cut', 'color', 'clarity']

# df.iloc[:5,]
# df.loc[:,['cut', 'color', 'clarity']].describe()

# atrenkame 5 proc., kad skaičiavimai netruktų per ilgai
df = df.groupby('cut', group_keys=False).apply(pd.DataFrame.sample,random_state=0, frac=.05)

X0, X1, y = df.loc[:,qCols].values,df.loc[:,['color','clarity']].values, df['cut'].values

X0_train, X0_test, X1_train, X1_test, y_train, y_test = train_test_split(X0, X1, y, train_size=0.75,stratify=y, random_state=0)
X0.shape,X1.shape

# Duomenų rinkmenoje diamonds.csv pateikiami duomenys apie deimantus, paimti iš R paketo ggplot2. Stulpelių aprašymas:

# price - kaina USD;

# carat - svoris karatais; 

# cut - pjovimo kokybė (Fair, Good, Very Good, Premium, Ideal);

# color - spalva (kinta nuo D (geriausia) iki J (blogiausia));

# clarity - skaidrumas (I1 (prasčiausias), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (geriausias));

# x - ilgis mm (0–10.74);

# y - plotis mm (0–58.9);

# z - gylis mm (0–31.8);

# depth - išvestinis dydis = 2 * z / (x + y);

# table - deimanto viršutinės dalies santykinis plotis.

# cut, color, clarity - kokybiniai požymiai, likę - kiekybiniai požymiai.

((2697, 7), (2697, 2))

In [ ]:
import numpy as np

# 1

In [36]:
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

## a)

In [38]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X0_train, y_train)
gnbpred = gnb.predict(X0_test)
print(f1_score(y_test, gnbpred, average='macro'), jaccard_score(y_test, gnbpred, average="micro"))

0.4652325676770775 0.43312101910828027


## b)

In [21]:
from sklearn.preprocessing import OrdinalEncoder
oenc = OrdinalEncoder()
oenc.fit(X1)

OrdinalEncoder()

In [39]:
from sklearn.naive_bayes import CategoricalNB
cnb = CategoricalNB()
cnb.fit(oenc.transform(X1_train), y_train)
cnbpred = cnb.predict(oenc.transform(X1_test))
print(f1_score(y_test, cnbpred, average='macro'), jaccard_score(y_test, cnbpred, average="micro"))

0.15950654752696725 0.25348189415041783


## c)

In [50]:
from sklearn.model_selection import GridSearchCV
par_grid = {'C': [0.01,0.1, 1.0, 10, 100, 1000]}

In [60]:
gsf1 = GridSearchCV(estimator=SVC(kernel='rbf' , decision_function_shape='ovr'), 
                           param_grid=par_grid, 
                           scoring='f1_macro',cv=10)
gsf1.fit(X0_train,y_train)
gsf1pred = gsf1.best_estimator_.predict(X0_test)
print(f1_score(y_test, gsf1pred, average='macro'))

0.18322059829051962


In [61]:
gsjm = GridSearchCV(estimator=SVC(kernel='rbf' , decision_function_shape='ovr'), 
                           param_grid=par_grid, 
                           scoring='jaccard_micro',cv=10)
gsjm.fit(X0_train,y_train)
gsjmpred = gsjm.best_estimator_.predict(X0_test)
print(jaccard_score(y_test, gsjmpred, average='micro'))

0.2605042016806723


Geriausiai prognozavo Gauso NB pagal kiekybinius kintamuosius

Galimai nes kiekybiniai kintamieji yra issideste pagal gauso pasiskirstyma

# 2

In [158]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [159]:
gamk_grid = {
            'learning_rate':[0.1, 0.5, 1.0],
            'n_estimators':[5,10,15,20],
            'min_samples_split': [2, 4, 6]}

In [160]:
gamk = GridSearchCV(estimator=GradientBoostingClassifier(), 
                           param_grid=gamk_grid, 
                           scoring='f1_micro',cv=10)
gamk.fit(X0_train, y_train)

GridSearchCV(cv=10, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 0.5, 1.0],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [5, 10, 15, 20]},
             scoring='f1_micro')

In [161]:
gamk.best_estimator_

GradientBoostingClassifier(min_samples_split=6, n_estimators=20)

In [162]:
print("Klasifikavimo metrikos testavimo aibėje: \n{}\n".format(classification_report(gamk.best_estimator_.predict(X0_test),y_test)))

Klasifikavimo metrikos testavimo aibėje: 
              precision    recall  f1-score   support

        Fair       0.75      0.75      0.75        20
        Good       0.51      0.70      0.59        44
       Ideal       0.94      0.81      0.87       315
     Premium       0.81      0.64      0.71       220
   Very Good       0.30      0.59      0.40        76

    accuracy                           0.72       675
   macro avg       0.66      0.70      0.66       675
weighted avg       0.79      0.72      0.74       675




Matome iš rezulatų lentelės kad modelis nėra persimokęs.

# 3

In [167]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [172]:
gnb = GaussianNB()
jaccavk = SVC(C=1000, probability = True)
hard = VotingClassifier([('Gaussian NB',gnb),('Jaccard AVK',jaccavk)], voting="hard")
soft = VotingClassifier([('Gaussian NB',gnb),('Jaccard AVK',jaccavk)], voting="soft",
                    weights=[0.43312101910828027, 0.2605042016806723])

gnb.fit(X0_train, y_train)
jaccavk.fit(X0_train, y_train)
hard.fit(X0_train, y_train)
soft.fit(X0_train, y_train)

for clf, label in zip([gnb, gsjm, hard, soft], ['Gaussian NB', 'Jaccard AVK', 'Hard', 'Soft']):
        print("Accuracy: ", accuracy_score(clf.predict(X0_test), y_test))

Accuracy:  0.6044444444444445
Accuracy:  0.41333333333333333
Accuracy:  0.49333333333333335
Accuracy:  0.5881481481481482
